<b>Задание:</b> Необходиом разработать модель предсказания дефолта. Даны финансовые данные 32 395 компаний (выручка, активы, пассивы и т.д). Результат будет оцениваться на скрытой выборке из 200 компаний по метрике качества accuracy_score


<b>Метрика качества</b> accuracy_score вычисляется, как ( True positive + True negative ) / Total.

## Ник vladislareon 
#### Лучший результат 0.645.

### Импортируем библиотеки

In [149]:
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder


### Подгружаем данные

In [2]:
%%bash

wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-train.csv
wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-test.csv
wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-desc.csv

--2022-04-29 15:57:08--  https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5329268 (5.1M) [text/plain]
Saving to: вЂ�PD-data-train.csvвЂ™

     0K .......... .......... .......... .......... ..........  0% 1.26M 4s
    50K .......... .......... .......... .......... ..........  1% 1.15M 4s
   100K .......... .......... .......... .......... ..........  2% 1.68M 4s
   150K .......... .......... .......... .......... ..........  3% 1.68M 3s
   200K .......... .......... .......... .......... ..........  4% 1.82M 3s
   250K .......... .......... .......... .......... ..........  5% 1.83M 3s
   300K .......... .......... .......... .......... ..........  6% 1.87M 3s
   350K ..........

In [2]:
train_df = pd.read_csv('PD-data-train.csv', sep=';')
test_df = pd.read_csv('PD-data-test.csv', sep=';')
desc_df = pd.read_csv('PD-data-desc.csv', sep=';')

Описание данных

Количество разных типов фичей

In [3]:
train_df.dtypes.value_counts()

float64    25
int64      11
object      1
dtype: int64

Уровень дефолт-рейта в выборке = 6,7219%

In [4]:
train_df['default_12m'].values.mean()

0.06454699799351751

In [5]:
train_df.record_id.nunique()

32395

32395 уникальных наблюдений

In [6]:
train_df.columns

Index(['record_id', 'ar_revenue', 'ar_total_expenses', 'ar_sale_cost',
       'ar_selling_expenses', 'ar_management_expenses', 'ar_sale_profit',
       'ar_balance_of_rvns_and_expns', 'ar_profit_before_tax', 'ar_taxes',
       'ar_other_profit_and_losses', 'ar_net_profit', 'ab_immobilized_assets',
       'ab_mobile_current_assets', 'ab_inventory', 'ab_accounts_receivable',
       'ab_other_current_assets', 'ab_cash_and_securities', 'ab_losses',
       'ab_own_capital', 'ab_borrowed_capital', 'ab_long_term_liabilities',
       'ab_short_term_borrowing', 'ab_accounts_payable', 'ab_other_borrowings',
       'bus_age', 'ogrn_age', 'adr_actual_age', 'head_actual_age',
       'cap_actual_age', 'ul_staff_range', 'ul_capital_sum', 'ul_founders_cnt',
       'ul_branch_cnt', 'ul_strategic_flg', 'ul_systematizing_flg',
       'default_12m'],
      dtype='object')

### Заполним пропуски средними значениями признаков.
### Эксперементы  с заполнением нулями показывали худший результат.

In [7]:
train_df_clear = train_df.fillna(train_df.mean())

C:\Users\vladi\AppData\Local\Temp/ipykernel_9012/528861120.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_df_clear = train_df.fillna(train_df.mean())


### Удалим признак, состоящий исключительно из 0:

In [8]:
train_df_clear.drop(['ul_systematizing_flg'], axis=1, inplace= True)

In [9]:
test_df_clear = test_df.fillna(test_df.mean())
test_df_clear.drop(['ul_systematizing_flg'], axis=1, inplace= True)

C:\Users\vladi\AppData\Local\Temp/ipykernel_9012/3333363208.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test_df_clear = test_df.fillna(test_df.mean())


In [13]:
train_df_clear_new = train_df.dropna()
train_df_clear_new.drop(['ul_systematizing_flg'], axis=1, inplace= True)

D:\conda\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [148]:
train_df_clear.corr()['default_12m']

record_id                      -0.002880
ar_revenue                     -0.036943
ar_total_expenses              -0.034928
ar_sale_cost                   -0.035301
ar_selling_expenses            -0.012747
ar_management_expenses         -0.005908
ar_sale_profit                 -0.021720
ar_balance_of_rvns_and_expns    0.003369
ar_profit_before_tax           -0.020708
ar_taxes                       -0.027337
ar_other_profit_and_losses     -0.007286
ar_net_profit                  -0.019033
ab_immobilized_assets          -0.016554
ab_mobile_current_assets       -0.026648
ab_inventory                   -0.025796
ab_accounts_receivable         -0.017590
ab_other_current_assets        -0.005771
ab_cash_and_securities         -0.010921
ab_losses                      -0.035023
ab_own_capital                 -0.038136
ab_borrowed_capital            -0.015753
ab_long_term_liabilities       -0.003993
ab_short_term_borrowing        -0.007287
ab_accounts_payable            -0.016237
ab_other_borrowi

In [15]:
train_df['ul_staff_range'].value_counts()

[1-100]      29281
(100-500]     2757
> 500          357
Name: ul_staff_range, dtype: int64

### В силу того, что признаки без пропусков имеют больщую корреляцию с таргетом, обучим 2 классификатора, соответственно для объектов без пропусков и для объектов с пропусками, но только с полными признаками:

**Идея показала не лучшие результаты.**

In [17]:
good_columns = train_df.dropna(axis=1).columns.drop('default_12m')
bad_columns = list(set(train_df.columns) - set(good_columns))

In [350]:
X_train, X_test, y_train, y_test = train_test_split(train_df.drop('default_12m', axis=1),
                                                    train_df['default_12m'],
                                                    random_state=42, shuffle=True)
full_index_train = list(set(X_train.index) & set(train_df.dropna().index))
full_index_test = list(set(X_test.index) & set(test_df.dropna().index))

null_index_train = list(set(X_train.index) - set(full_index_train))
null_index_test = list(set(X_test.index) - set(full_index_test))

In [351]:
X_train_full, y_train_full = X_train.loc[full_index_train], y_train.loc[full_index_train]
X_test_full, y_test_full = X_test.loc[full_index_test], y_test.loc[full_index_test]

X_train_null, y_train_null = X_train.loc[null_index_train, good_columns], y_train.loc[null_index_train]
X_test_null, y_test_null = X_test.loc[null_index_test, good_columns], y_test.loc[null_index_test]

In [352]:
column_transformer = ColumnTransformer([
    ('ohe',  OneHotEncoder(handle_unknown="ignore"), ['ul_staff_range'])
], remainder='passthrough')
X_train_full =column_transformer.fit_transform(X_train_full)
X_test_full=column_transformer.fit_transform(X_test_full)
X_train_null =column_transformer.fit_transform(X_train_null)
X_test_null=column_transformer.fit_transform(X_test_null)

In [353]:
model = CatBoostClassifier( **{'logging_level' : 'Silent'})
model.fit(X_train_full, y_train_full)
preds_full = model.predict_proba(X_test_full)

In [354]:
model = CatBoostClassifier( **{'logging_level' : 'Silent'})
model.fit(X_train_null, y_train_null)
preds_null = model.predict_proba(X_test_null)

In [355]:
preds = y_test.copy()
preds[full_index_test] = preds_full[:, 1]
preds[null_index_test] = preds_null[:, 1]
roc_auc_score(y_test, preds)

0.6658264245295433

In [21]:
X = train_df_clear.drop('default_12m', axis=1)
y = train_df_clear['default_12m']
column_transformer = ColumnTransformer([
    ('ohe',  OneHotEncoder(handle_unknown="ignore"), ['ul_staff_range'])
], remainder='passthrough')
X = column_transformer.fit_transform(X)
y.value_counts()

0    30304
1     2091
Name: default_12m, dtype: int64

### В силу несбаланированности таргета сделаем оверсэмплинг при помощи библиотеки ```SMOTE```.

**Эксперимент оказался неудачным на тестовых данных.**

In [22]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [23]:
y.value_counts()

0    30304
1    30304
Name: default_12m, dtype: int64

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42, shuffle=True, stratify=y)

In [49]:
best_feat = pd.DataFrame()
best_feat['ind'] = X_train.shape[1] * [0]
for i in range(X_train.shape[1]):
    best_feat['ind'].iloc[i] = t(X_train[:, i], y_train)
best_feat = list(best_feat.sort_values(by = 'ind', ascending=False).head(32).index)
tr =  Best_weight(best_feat)
X_train_lr = tr.transform(X_train)
X_test_lr = tr.transform(X_test)

In [50]:
model = CatBoostClassifier( **{'logging_level' : 'Silent'})
model.fit(X_train_lr, y_train)
preds = model.predict_proba(X_test_lr)
roc_auc_score(y_test, preds[:, 1])

0.9832665647729895

### Валидация результата

In [157]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
X_train, X_test, y_train, y_test = train_test_split(train_df_clear.drop('default_12m', axis=1),
                                                    train_df_clear['default_12m'], shuffle=True, test_size=0.15,
                                                   stratify= train_df_clear['default_12m'], random_state=142)

column_transformer = ColumnTransformer([
    ('ohe',  OneHotEncoder(handle_unknown="ignore"), ['ul_staff_range'])
], remainder='passthrough')
X_train = column_transformer.fit_transform(X_train)
X_test = column_transformer.fit_transform(X_test)

### Отбор признаков с помощью линейной модели:

In [158]:
lr = LogisticRegression(max_iter=100000).fit(X_train, y_train)

In [159]:
modul = np.abs(lr.coef_)
best_weight = list(pd.DataFrame(modul).T.sort_values(by = 0, ascending=False).head(32).index)

In [160]:
class Best_weight():
    def __init__(self, best_weight):
        self.best_weight = best_weight
    def fit(self, _data1, y=None):
        return self
    def transform(self, data1, y=None):
        data = data1.copy()
        data = data[:, self.best_weight]
        return data   

In [161]:
tr =  Best_weight(best_weight)
X_train_lr = tr.transform(X_train)
X_test_lr = tr.transform(X_test)

In [162]:
model = CatBoostClassifier( **{'logging_level' : 'Silent'})
model.fit(X_train_lr, y_train)
preds = model.predict_proba(X_test_lr)
print(roc_auc_score(y_test, preds[:, 1]))

0.71836443321069


### Методы фильтрации:

In [163]:
def t(data1, y):
    data = data1.copy()
    mu_plus = data[y == 1].mean()
    mu_minus = data[y == 0].mean()
    sig_plus = data[y == 1].std()
    sig_minus = data[y == 0].std()
    n_plus = data[y == 1].shape[0]
    n_minus = data[y == 0].shape[0]
    return np.abs(mu_plus - mu_minus) / np.sqrt((n_plus * sig_plus ** 2 + n_minus * sig_minus ** 2) / (n_plus + n_minus))


In [164]:
best_feat = pd.DataFrame()
best_feat['ind'] = X_train.shape[1] * [0]
for i in range(X_train.shape[1]):
    best_feat['ind'].iloc[i] = t(X_train[:, i], y_train)

best_feat = list(best_feat.sort_values(by = 'ind', ascending=False).head(32).index)

In [165]:
tr =  Best_weight(best_feat)
X_train_lr = tr.transform(X_train)
X_test_lr = tr.transform(X_test)

In [167]:
model = CatBoostClassifier(logging_level = 'Silent')
model.fit(X_train_lr, y_train)
preds = model.predict_proba(X_test_lr)
print(roc_auc_score(y_test, preds[:, 1]))

0.7358624926792223


### Попробуем закодировать категориальный признак при помощи счетчиков:

In [33]:
class Meantargetencoding_smoothing(BaseEstimator, TransformerMixin):
    def __init__(self, categorical, C):
        self.C = C
        self.categorical = categorical
    def fit(self, data, y):
        self.mean_cat = {key: None for key in self.categorical}
        data['y'] = y
        for categoria in self.mean_cat:
            self.mean_cat[categoria] = ((data.groupby(categoria)['y'].sum() +
                                         self.C * data['y'].mean()) /
                           (data.groupby(categoria)['y'].size() +
                            self.C))
        data.drop(columns=['y'], inplace=True)
        return self
    def transform(self, data1):
        data = data1.copy()
        for cat, tar in self.mean_cat.items():
            data[cat].replace(tar, inplace=True)
        return data   

In [34]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
X_train1, X_test1, y_train1, y_test1 = train_test_split(train_df_clear.drop('default_12m', axis=1),
                                                    train_df_clear['default_12m'],
                                                    random_state=42, shuffle=True)

coder = Meantargetencoding_smoothing(['ul_staff_range'], 20).fit(X_train1, y_train1)
X_test1 = coder.transform(X_test1)
X_train1 = coder.transform(X_train1)

In [35]:
model = CatBoostClassifier( **{'logging_level' : 'Silent'})
model.fit(X_train1, y_train1)
preds = model.predict_proba(X_test1)
roc_auc_score(y_test1, preds[:, 1])

0.7089923674167653

# Самыми удачными решениями оказались:

* Заполнение пропущенных значений средними значениями.

* Использование ```OneHotEncoder``` на категориальном признаке.

*  В силу раздутия признакового пространства отбор фичей после трансформаций при помощи  t-статистики.

* Обучение градиентного бустинга ```Catboost``` (так как в тестовой выборке нет сильной несбалонсированности, перевзвешивание классов таргета с помощью ```auto_class_weights``` делало только хуже) и подбор гиперпараметров при помощи библиотеки ```optuna``` и метрики ```ROC-AUC``` (не смотря на несбалансированность классов использование ```AUC-PR``` показало худший результат).





**Ник vladislareon**

**Лучший результат 0.645**

In [101]:
import optuna
def objective(trial):
    params = {
        'iterations' : trial.suggest_int('iterations', 50, 280),                         
        'depth' : trial.suggest_int('depth', 4, 16),                                                      
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 3, 280),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'learning_rate' :trial.suggest_loguniform('learning_rate', 0.01, 0.2),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter'])
    }
    model = CatBoostClassifier( **{'logging_level' : 'Silent'}, **{**params})        
    model.fit(X_train_lr, y_train)
    preds = model.predict_proba(X_test_lr)
    now_score = roc_auc_score(y_test, preds[:, 1])
    return now_score
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = int(20))

[I 2022-05-06 20:51:48,690] A new study created in memory with name: no-name-f0e8d1fb-2217-483e-8920-b7f10948101d
[I 2022-05-06 20:51:53,713] Trial 0 finished with value: 0.7270467352834857 and parameters: {'iterations': 148, 'depth': 9, 'random_strength': 52, 'early_stopping_rounds': 131, 'bagging_temperature': 0.07550027026973584, 'learning_rate': 0.1443309479642427, 'od_type': 'Iter'}. Best is trial 0 with value: 0.7270467352834857.
[I 2022-05-06 20:52:20,737] Trial 1 finished with value: 0.712906776027641 and parameters: {'iterations': 81, 'depth': 12, 'random_strength': 38, 'early_stopping_rounds': 202, 'bagging_temperature': 30.158739776691572, 'learning_rate': 0.01975436880966701, 'od_type': 'Iter'}. Best is trial 0 with value: 0.7270467352834857.
[I 2022-05-06 20:52:23,625] Trial 2 finished with value: 0.7095111962360695 and parameters: {'iterations': 194, 'depth': 6, 'random_strength': 37, 'early_stopping_rounds': 25, 'bagging_temperature': 1.3638837167966686, 'learning_rate':

In [102]:
from sklearn.metrics import accuracy_score
model = CatBoostClassifier( **{'logging_level' : 'Silent'},**{**study.best_trial.params})
model.fit(X_train_lr, y_train)
preds = model.predict_proba(X_test_lr)
roc_auc_score(y_test, preds[:, 1])

0.7311088911368853

In [116]:
cutoff = train_df_clear['default_12m'].mean()

In [119]:
predict =  (model.predict_proba(X_test_lr)[:,1]>cutoff).astype(int)

In [112]:
accuracy_score(y_test, predict)

0.6699588477366255

In [153]:
model = CatBoostClassifier( **{'logging_level' : 'Silent'})#,  **{'auto_class_weights' : 'Balanced'})
model.fit(X_train, y_train)
preds = model.predict_proba(X_test)
roc_auc_score(y_test, preds[:, 1])

0.7258845231292942

### Построение модели на всех данных

In [121]:
column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown="ignore"), ['ul_staff_range'])
], remainder='passthrough')
X_train_all =column_transformer.fit_transform(train_df_clear.drop('default_12m', axis=1))
X_test_all =column_transformer.fit_transform(test_df_clear)

tr =  Best_weight(best_feat)
X_train_all = tr.transform(X_train_all)
X_test_all = tr.transform(X_test_all)


In [122]:
model = CatBoostClassifier( **{'logging_level' : 'Silent'},**{**study.best_trial.params},  **{'auto_class_weights' : 'Balanced'})
model.fit(X_train_all, train_df_clear['default_12m'])

### Формирование файла с ответом

In [129]:
answ_df = test_df[['record_id']]

In [130]:
answ_df['id'] = answ_df['record_id']

C:\Users\vladi\AppData\Local\Temp/ipykernel_9012/3997193366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answ_df['id'] = answ_df['record_id']


In [131]:
answ_df.drop('record_id',axis=1, inplace=True)

D:\conda\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [132]:
cutoff = train_df_clear['default_12m'].mean()

In [145]:
answ_df['predict'] = (model.predict_proba(X_test_all)[:,1]>cutoff).astype(int)

C:\Users\vladi\AppData\Local\Temp/ipykernel_9012/3541171757.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answ_df['predict'] = (model.predict_proba(X_test_all)[:,1]>cutoff + 0.3).astype(int)


In [147]:
answ_df.to_csv('PD-submit15.csv',index=False, sep=';')